In [ ]:
import pickle
import os
import math
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.spatial import distance
import matplotlib.pyplot as plt
np.random.seed(42)
import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Dense, Dropout, Input, Concatenate, concatenate, Flatten, Reshape, Lambda
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from sklearn.model_selection import train_test_split as splt
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from bokeh.plotting import figure, output_notebook, show, output_file, save
from bokeh.io import export_png

#output_notebook()

In [ ]:
from keras import backend as K

def get_activations(model, layer, X_batch):
    activations_f = K.function([model.layers[0].input, K.learning_phase()], [layer.output,])
    activations = activations_f((X_batch, False))
    return activations

In [ ]:
def step1(age, sex, chd=True):
    # CHD
    if chd:  # constants for chd
        if sex:  # 0 for men and 1 for women
            a = -29.8
            p = 6.36
        else:
            a = -22.1
            p = 4.71

    else:  # constants for non chd
        if sex:
            a = -31.0
            p = 6.62
        else:

            a = -26.7
            p = 5.64

    # print("a =", a, "; p =",p)

    s = math.exp(-(math.exp(a)) * (age - 20) ** p)
    return s


def step2(chol, SBP, smoker, chd=True):
    if chd:
        c_smoker = 0.71
        c_chol = 0.24
        c_SBP = 0.018
    else:
        c_smoker = 0.63
        c_chol = 0.02
        c_SBP = 0.022

    w = (c_chol * (chol - 6)) + (c_SBP * (SBP - 120)) + (c_smoker * smoker)
    return w


def score_algorithm(age, chol, SBP, sex, smoker):
    # CHD
    smoker = 1 if smoker >= 1 else 0

    
    s = step1(age, sex)
    s10 = step1(age + 10, sex)

    w = step2(chol, SBP, smoker)
    

    s = s ** (math.exp(w))
    s10 = s10 ** (math.exp(w))
    try:
        stot = s10 / s
    except:
        stot = 1
    riskc = 1 - stot

    # NON CHD
    s = step1(age, sex, chd=False)
    s10 = step1(age + 10, sex, chd=False)

    w = step2(chol, SBP, smoker, chd=False)

    s = s ** (math.exp(w))
    s10 = s10 ** (math.exp(w))
    try:
        stot = s10 / s
    except:
        stot = 1
    risknon = 1 - stot

    # print ("risk CHD: ", riskc *100)
    # print ("risk nonCHD: " ,risknon * 100)
    risktot = 1 - (1 - riskc) * (1 - risknon)

    # print('total RISK:',risktot)
    return risktot

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
dfx = pd.read_csv("Data/plic_milano_clean.csv")

In [ ]:
y = np.zeros(dfx.shape[0])
for i, visit in dfx.iterrows():
    age = dfx['ana:age'][i]
    chol = dfx['lab:total_cholesterol'][i] * 0.02586
    pas = float(dfx['esa_obi:sbp'][i])
    sex = dfx['ana:gender'][i]
    smoker = dfx['ana_fis:smoking_recod'][i]
    
    y[i] = score_algorithm(age, chol, pas, sex, smoker)

# Thresholds are 0 - 0.01, 0.01 - 0.02, 0.02 - 0.05, 0.05 - 1    

y_cat = np.zeros((y.shape[0], 4))
stats = np.zeros(y.shape[0])
for i, val in enumerate(y):
    if(val < 0.01): # No
        y_cat[i][0] = 1
    elif(val < 0.02): # Low
        y_cat[i][1] = 1
        stats[i] = 1
    elif(val < 0.05): # Medium
        y_cat[i][2] = 1
        stats[i] = 2
    else:             # High 
        y_cat[i][3] = 1
        stats[i] = 3
        
y_cat.shape

plt.hist(stats)
plt.show()

unique, counts = np.unique(stats, return_counts=True)
for i in range(4):
    print("El: {} \tCount: {} \tPercentage: {}%".format(int(unique[i]), counts[i], round(counts[i]/len(stats)*100, 2)))

In [ ]:
dfx = dfx.drop(labels = ['lab:total_cholesterol', "esa_obi:sbp", "ana_fis:smoking_recod", "Unnamed: 0"], axis=1)
dfx = dfx.dropna(how='any', axis=1)
dfx = dfx.select_dtypes(include=['int'])

In [ ]:
threshold = 10
dfxInt = dfx.select_dtypes(include=['int'])
dfxCat = dfxInt[dfxInt.columns[dfxInt.max()<=threshold]]

#dfxCont = dfxInt[dfxInt.columns[dfxInt.max()>threshold]] + dfx.select_dtypes(include=['float'])

# dfxCat.describe()
# dfxCont.describe()
print(dfx.shape)
print(dfxCat.shape)

In [ ]:
headers = set(col.split(':')[0] for col in dfxCat.columns.values)

In [ ]:
headers

In [ ]:
drop_cols = dfxCat.columns[dfxCat.columns.str.endswith('_self')].values.tolist()

In [ ]:
drop_cols

In [ ]:
dfxCat = dfxCat.drop(drop_cols, axis=1)

In [ ]:
categories = dfxCat[dfxCat.columns[dfxCat.columns.str.startswith('ana_pat') |  
                                   dfxCat.columns.str.startswith('ana_far') | 
                                   dfxCat.columns.str.startswith('ult_tsa')]]

In [ ]:
categories_np = categories.values
# categories_np = np.transpose(categories_np)
categories_np.shape

In [ ]:

models = []
for i, cat in enumerate(categories):
    m = Sequential()
    m.add(Embedding(10, categories[cat].nunique(), input_length=1, input_shape=()))
    m.add(Flatten())
    models.append(m)
    print(model.summary())
    
full_model = Sequential()
full_model.add(Concatenate(models))
full_model.add(Dense(32, activation='sigmoid'))
full_model.add(Dense(16, activation='sigmoid'))
full_model.add(Dense(8, activation='sigmoid'))
full_model.add(Dense(4, activation='softmax'))    



# catIn = Input(shape=(categories_np.shape[1],))
# models = []
# for i, cat in enumerate(categories):
#     m = Embedding(10, categories[cat].nunique(), input_length=1)(catIn)
#     m = Flatten()(m)
#     models.append(m)





# catInputs = []
# for i in range(categories_np.shape[0]):
#     catInputs.append(Input(shape=(1,)))
# models = []
# for i, cat in enumerate(categories):
#     m = Embedding(10, categories[cat].nunique(), input_length=1)(catInputs[i])
#     m = Flatten()(m)
#     models.append(m)
    

    
    
    
# concatenation = Concatenate()(models)
# model = Dense(64, activation='sigmoid')(concatenation)
# model = Dense(32, activation='sigmoid')(model)
# model = Dense(16, activation='sigmoid')(model)
# model = Dense(8, activation='sigmoid')(model)

# output = Dense(4, activation='softmax')(model)

# model = Model(inputs = catInputs, outputs = [output])

In [ ]:
full_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
full_model.summary()

In [ ]:
x_Cat_tr, x_Cat_ts, y_tr, y_ts = splt(categories_np, y_cat, test_size=0.25, random_state=42)

In [ ]:
history = model.fit(x=x_Cat_tr, y=y_tr, 
                    validation_data=(x_Cat_ts, y_ts), 
                    epochs=100
                   )

In [ ]:
plot_history(history)

In [ ]:
#fn = "_embedded_single_cat_"

In [ ]:
#model.save("Models/model" + fn + ".h5")

In [ ]:
#pickle.dump(history, open("History/history" + fn + ".pkl", "wb"))

In [ ]:
%matplotlib inline

In [ ]:
import keras
import pydot as pyd
from IPython.display import Image
from keras.utils.vis_utils import model_to_dot

keras.utils.vis_utils.pydot = pyd

#Visualize Model

def visualize_model(model):
  return Image(model_to_dot(model).create(prog='dot', format='png'))

visualize_model(model)

In [ ]:
embeddings_tr = get_activations(model, model.layers[1], X_batch=x_Cat_tr)[0]
embeddings_ts = get_activations(model, model.layers[1], X_batch=x_Cat_ts)[0]

In [ ]:
len(model.layers)

In [ ]:
model.layers

In [ ]:
embeddings_tr.shape

In [ ]:
embeddings_ts.shape

In [ ]:
embeddings_tr = embeddings_tr.reshape((4335, 2, 48))
embeddings_ts = embeddings_ts.reshape((1445, 2, 48))

In [ ]:
colors_map_tr = np.argmax(y_tr, axis=1)
colors_map_ts = np.argmax(y_ts, axis=1)

In [ ]:
scale = 1

In [ ]:
colors = [x for x in 'yellow-orange-red-blue'.split('-')]

In [ ]:
from sklearn.manifold import MDS, TSNE

In [ ]:
embeddings_tr_tsne_list = []
for layer in range(embeddings_tr.shape[2]):
    print(layer, end=' ')
    tsne = TSNE(n_components=2)
    embeddings_tr_tsne_list.append(tsne.fit_transform(embeddings_tr[:, :, layer]))

In [ ]:
len(embeddings_tr_tsne_list)

In [ ]:
for layer in range(embeddings_tr.shape[2]):
    print(layer)
    p = figure(plot_width=300, plot_height=300)
    for cl in range(4):
        indices = np.where(colors_map_tr==cl)[0]
        p.circle(embeddings_tr_tsne_list[layer][indices, 0]*scale*100, embeddings_tr_tsne_list[layer][indices, 1]*scale*100, 
                 color=colors[cl], size=20, alpha=0.4)
    output_file("bokeh1_" + str(layer) + ".html")
    save(p)
    #show(p)
    

In [ ]:
# p = figure(plot_width=600, plot_height=600)

# for cl in range(4):
#     indices = np.where(colors_map_ts==cl)[0]
#     p.circle(embeddings_ts_tsne[indices, 0]*scale*100, embeddings_ts_tsne[indices, 1]*scale*100, 
#              color=colors[cl], size=20, alpha=0.4)

# show(p)